In [3]:
import json
from uuid import uuid4
from collections import Counter

import pandas as pd
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import optuna
import shap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, f1_score

from imblearn.over_sampling import RandomOverSampler

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

import pickle


/home/john/miniconda3/envs/ds/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
results_df = pd.read_csv('data/ner_deep_learning_results.csv')
mistakes = results_df[(results_df['y_pred'] == 0) & (results_df.model == 'dslim/bert-base-NER')]['Name'].to_numpy()
mistakes
rng = np.random.default_rng(seed=0)
hard_name = rng.choice(mistakes)
print(hard_name)
switch_name = 'Peter'
targets = ['no emotion', 'anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']


Riku


# NER False Negatives

array(['A', 'A', 'A', ..., 'Zion', 'Zion', 'Zion'], dtype=object)

# feature importance

In [4]:
model_name='dslim/bert-base-NER'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)



def train(who='all', n_samples=100, fp='data/output.pkl'):
    """
    who: 'all' - anonyimise everyone
    who: 'rare-once' - only insert the rare name once
    """

    removed_names = set()

    def anonymise(sentence, who='all'):
        global rare_insert_count
        nlp = pipeline("ner", model=model, tokenizer=tokenizer)
        ner_results = nlp(sentence)
        for item in ner_results:
            # https://huggingface.co/dslim/bert-base-NER
            if item['entity'] in ['B-PER', 'I-PER']:
                # Assume that there rare name IS in the dataset. We replace another name with hard name.
                if who == 'all':
                    if item['word'] == switch_name:
                        sentence = sentence.replace(item['word'], hard_name)
                        print(sentence)
                    else:
                        sentence = sentence.replace(item['word'], '')
                        removed_names.add(item['word'])

                elif who == 'rare_once':
                    if item['word'] == switch_name and rare_insert_count < 1:
                        sentence = sentence.replace(item['word'], hard_name)
                        print(sentence)
                        rare_insert_count += 1
                    else:
                        sentence = sentence.replace(item['word'], '')
                        removed_names.add(item['word'])
                elif who == 'noone':
                    if item['word'] == switch_name:
                        sentence = sentence.replace(item['word'], hard_name)
                        print(sentence)
                else:
                    raise NotImplementedError(f'{who} is an unknown option')

        return sentence

    def process(split='train', ner=True):    
    
        utterance = []
        ids = []
        label = []
        act = []
        
        # Apply the function to all examples in the dataset
        dataset = load_dataset('daily_dialog', split=split)
        
        if n_samples:
            nd = n_samples
        else:
            nd = len(dataset)
        
        for i in tqdm(range(nd)):
            example = dataset[i]
            did = uuid4()
            for j in range(len(example['dialog'])):
                text = example['dialog'][j]
                # add previous sentnce xontext
                if j > 1:
                    text = str(example['emotion'][j - 1]) + ' ' + example['dialog'][j - 1] + ' ' + text
                if ner:
                    text = anonymise(text, who=who)
                utterance.append(text)
                act.append(example['act'][j])
                label.append(example['emotion'][j])
                ids.append(did)

        data = {
            'text': utterance,
            'label': label,
            'attr': act,
            'id': ids
        }

        df = pd.DataFrame(data=data)

        return df
    
    global rare_insert_count
    rare_insert_count = 0
    df_train = process(split='train')
    print('n train', len(df_train))
    rare_insert_count = 0
    df_valid = process(split='validation')
    rare_insert_count = 0
    df_test = process(split='test')

    print(list(set(removed_names)))

    # improves macro f1
    rus = RandomOverSampler(random_state=42)
    df_train, _ = rus.fit_resample(df_train, df_train.label)

    counts = Counter(df_train.label)
    print('train label dist.', counts)

    clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=1.0930076764057076e-05, n_jobs=-1)
    vec = TfidfVectorizer()

    X_train_tfidf = vec.fit_transform(df_train.text.to_list())
    X_valid_tfidf = vec.transform(df_valid.text.to_list())
    X_test_tfidf = vec.transform(df_test.text.to_list())

    clf.fit(X_train_tfidf, df_train.label)

    y_pred = clf.predict(X_test_tfidf)
    y_true = df_test.label
    report = classification_report(y_true, y_pred)
    print(report)

    r = (clf, vec, removed_names, X_train_tfidf, df_train, X_test_tfidf, df_test)

    f = open(fp, 'wb')
    pickle.dump(r, f)
    f.close()

    return r


NameError: name 'mistakes' is not defined

In [55]:
r_all = train(who='all', n_samples=None, fp='data/all.pkl') # anonymise everyone
r_rare = train(who='rare_once', n_samples=None, fp='data/rare_once.pkl') # anoymise everyone only inser the rare name once


Found cached dataset daily_dialog (/home/john/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd)


 Great idea ! Riku , I could use the drink . 
4  Great idea ! Riku , I could use the drink .   How about the new bar across road ? 


Did you hear Riku just retired ? 


Who broke the glass , Riku ? 


0  Well , so long ,  . Have fun .   You too , Riku . Give my regards to John . I'll see you in September . 


...

 55%|█████▌    | 6123/11118 [2:24:40<2:17:32,  1.65s/it] 

Riku , I hope you will accept my quotation for the air freshener . 
0  Then , I am sorry to say I must regretfully decline your offer and seek quotations for the other suppliers .   I'm sorry , Riku . I hope you'll continue to approach us on other occasions if you cannot accept our offer now . 
0  I'm sorry , Riku . I hope you'll continue to approach us on other occasions if you cannot accept our offer now .   Yes . I'll welcome other opportunities to do business with you . 


 55%|█████▌    | 6147/11118 [2:25:18<1:53:26,  1.37s/it]

Riku , listen to the lyrics of this song . 


 56%|█████▌    | 6216/11118 [2:27:44<2:49:04,  2.07s/it]

In [ ]:
r_noone = train(who='noone', n_samples=None, fp='data/noone.pkl') # anonymise nobody

# Evaluate label flipping

In [13]:
from scipy.stats import entropy
from sklearn.metrics import accuracy_score

def evaluate(fp):

    f = open(fp, 'rb')
    clf, vec, removed_names, X_train_tfidf, df_train, X_test_tfidf, df_test = pickle.load(f)
    f.close()

    explainer = shap.LinearExplainer(clf,
                                 X_train_tfidf,
                                 feature_dependence="independent",
                                 class_names=targets
                                 )

    probs  = clf.predict_proba(X_test_tfidf)

    # sort predictions by entropy descending order
    u = entropy(probs, axis=1)
    u_indx = np.argsort(u)[::-1]
    u_text = df_test.text.to_numpy()[u_indx]

    # print top 5 most informative predictions
    print(u_text[:5])

    # Names which NER removed but are not names
    blacklist = ['Mr', 'Call', 'Long', 'My', 'Car', 'He', 'Mum', 'Black', 'Drive', 'White', 'Ma', 'Z', 'B', 'Sun', '.'
        'Mad',
        'Mon',
        'Cat',
        'Ball',
        'Hell',
        'Sounds',
        'Shut',
        'Aunt',
        'Rich',
        'Map',
        'Kitty',
        'Dead',
        'God',
        'Uncle',
        'Don',
        'Pink',
        'Elvis',
        'Na',
        'Ad',
        'Smith',
        'Ward',
        'Lord',
        'Jin',
        'gun',
                 ]

    # Create a white list
    whitelist = []

    for name in removed_names:
        if '#' not in name:
            whitelist.append(name)

    setA = set(whitelist)
    setB = set(blacklist)

    # Get new set with elements that are only in a but not in b
    whitelist = list(setA.difference(setB))

    # Number of best predictions to check for label flipping
    top_pred = 3
    summary_plot = False

    names = ['', 'NAME', hard_name] + whitelist

    # top k highest entropy sentences
    entropy_k = 10
    sentences = u_text[:entropy_k]
    # predictions for all names, test sets
    preds = []

    for name in names:

        # add the name to start of every sentence
        test_sentences = [f'{name} {sentence}' for sentence in sentences]

        # get features
        test_sentences = vec.transform(test_sentences)

        # explain features
        shap_values = explainer.shap_values(test_sentences)

        X_test_array = test_sentences.toarray()

        if summary_plot:
            [print(test_sentence) for test_sentence in test_sentences[:5]]
            shap.summary_plot(shap_values,
                            X_test_array,
                            feature_names=vec.get_feature_names_out(),
                            class_names=targets)
        
        out = clf.predict(test_sentences)
        probs = clf.predict_proba(test_sentences)

        # get sorted label predictions
        best_k = np.argsort(probs, axis=1)[:, ::-1]

        preds.append(best_k)

    preds_empty = preds[0]
    preds_replace = preds[1]
    preds_names = preds[2:]

    accuracy = []

    # calculate accuracy
    for i, pred_names_i in enumerate(preds_names):

        # accuracy compared to ground truth for each sentence for one name
        accuracy_i = []

        # iterate over each test sentence
        for k in range(entropy_k):

            y_pred = pred_names_i[k][:top_pred]
            y_true = preds_empty[k][:top_pred]

            score = accuracy_score(y_true, y_pred)
            accuracy_i.append(score)
        
        accuracy.append(accuracy_i)

    mean_score = np.mean(accuracy, axis=1)

    data = {
        'Names': names[2:],
        'Metric': mean_score
    }

    df_flips = pd.DataFrame(data)
    df_flips = df_flips.sort_values('Metric').reset_index(drop=True)
    with pd.option_context('display.max_rows', None,
                        'display.max_columns', None,
                        'display.precision', 3,
                        ):
        print(df_flips)


In [14]:
evaluate('data/all.pkl')
evaluate('data/rare_once.pkl')
#evaluate('data/noone.pkl')

The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)


['6  I didn ’ t know it would be so big !   It is ! Look there , those are the tracks . And the jumping pit is over there . '
 '5  He ’ s the love of my life ! I ’ e really messed this up .   Come on , hon . Pull yourself together . It ’ s going to be alright . '
 '0  Did it scare you ?   Of course not . I just thought the movie was ... boring . '
 '2  That bad egg , who took the low road since he was a boy .   I think I must report to the cops . '
 "0  Oh , yes . I remember it clearly . Is there anything wrong with it ?   I'm afraid so . We found this crack on the bottom when I went back to my hotel . "]
            Names  Metric
0             Mad   0.067
1           Huang   0.167
2            Riku   0.200
3              OK   0.200
4             Min   0.200
5              Ok   0.200
6            Shan   0.200
7             Wei   0.200
8             Win   0.200
9          Mister   0.200
10            don   0.200
11          Gross   0.200
12           Coke   0.200
13            Due   0.2

The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)


            Names  Metric
0             Mad   0.100
1           Mouse   0.100
2             Sea   0.133
3          Mister   0.133
4             Win   0.133
5             Wei   0.133
6           Gross   0.133
7            Bull   0.167
8            What   0.167
9             Run   0.167
10           Bush   0.200
11           Bill   0.200
12           Shan   0.200
13             Li   0.200
14            Hey   0.200
15            Tai   0.200
16            Due   0.200
17        General   0.200
18        Darling   0.200
19           Wood   0.233
20          Huang   0.233
21         Dancer   0.233
22            Dog   0.233
23         Cruise   0.233
24             Ya   0.233
25           Twin   0.233
26           Wang   0.233
27            Kit   0.233
28           Over   0.233
29           Mark   0.267
30            Lin   0.267
31         Church   0.267
32          Brown   0.267
33          Miles   0.267
34      Christian   0.267
35          Child   0.267
36           Wall   0.267
37          

In [151]:
tex =  df_flips[:20].to_latex(
    index=False,
    float_format="{:.2f}".format,
    caption='hello')
print(tex)

\begin{table}
\caption{hello}
\begin{tabular}{lr}
\toprule
Names & Metric \\
\midrule
Yug & 0.50 \\
Julie & 0.70 \\
Yang & 1.00 \\
Stefan & 1.00 \\
Christine & 1.00 \\
Ted & 1.00 \\
Atlas & 1.00 \\
Po & 1.00 \\
Johnson & 1.00 \\
Tim & 1.00 \\
Chang & 1.00 \\
Bill & 1.00 \\
Vivian & 1.00 \\
Smith & 1.00 \\
Men & 1.00 \\
Mi & 1.00 \\
Nicole & 1.00 \\
Montgomery & 1.00 \\
G & 1.00 \\
Soo & 1.00 \\
\bottomrule
\end{tabular}
\end{table}



In [112]:
'#' in 'f#'

True